In [132]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time


X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


In [133]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [134]:
# For subject 1
X_train_valid_1 = X_train_valid[np.where(person_train_valid==0)[0]]
y_train_valid_1 = y_train_valid[np.where(person_train_valid==0)[0]] - 769
X_test_1 = X_test[np.where(person_test==0)[0]]
y_test_1 = y_test[np.where(person_test==0)[0]] - 769


In [135]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,X,Y):
        self.X = torch.FloatTensor(X)
        self.Y = torch.LongTensor(Y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.Y[index]

In [136]:
from sklearn.model_selection import train_test_split
X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(X_train_valid_1, y_train_valid_1,
                                                              test_size=0.2,shuffle=True)


X_train_1 = X_train_1.reshape(X_train_1.shape[0],1,X_train_1.shape[1],X_train_1.shape[2])
X_valid_1 = X_valid_1.reshape(X_valid_1.shape[0],1,X_valid_1.shape[1],X_valid_1.shape[2])
X_test_1 = X_test_1.reshape(X_test_1.shape[0],1,X_test_1.shape[1],X_test_1.shape[2])

In [137]:
train_set_1 = Dataset(X_train_1,y_train_1)
val_set_1 = Dataset(X_valid_1,y_valid_1)
test_set_1 = Dataset(X_test_1, y_test_1)

train_loader_1 = torch.utils.data.DataLoader(train_set_1,batch_size=32,shuffle=True)
val_loader_1 = torch.utils.data.DataLoader(val_set_1,batch_size = 12,shuffle=True)
test_loader_1 = torch.utils.data.DataLoader(test_set_1,batch_size=10,shuffle=True)

In [138]:
print(489/3)

163.0


In [139]:
'''
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()  # initial the model
        self.conv11 = nn.Conv2d(1,25,(1,10)) # depth 25, 22*990
        self.bn11 = nn.BatchNorm2d(25)
        # permute 990,  22 x 12
        self.conv12 = nn.Conv2d(25,25,kernel_size = (2,5),stride = (1,1)) # 25, 21 x 986
        self.bn12 = nn.BatchNorm2d(25) 
        
        self.conv13 = nn.Conv2d(25,25,kernel_size = (4,3),stride = (1,1)) # 25, 18 x 984
        self.bn13 = nn.BatchNorm2d(25) 
        
        self.conv14 = nn.Conv2d(25,25,kernel_size = (18,2),stride = (1,1)) # 25, 1 x 983
        self.bn14 = nn.BatchNorm2d(25) 
        # permute 0, 2,1,3 ----> N, 1,25,983
        self.pool1 = nn.MaxPool2d((1,3),stride = (1,3)) # 1, 25*328
        # now do normal conv - maxpool
        # block2
        # after permute, starts at 1,25,328
        self.conv21 = nn.Conv2d(1,50,kernel_size = (25,10),stride = (1,1)) # 50,1,319
        self.bn21 = nn.BatchNorm2d(50)
        self.pool21 = nn.MaxPool2d((1,3),stride = (1,3))
        # block3
        self.conv31 = nn.Conv2d(1,100,(50,10))# after permute 1, 100*150
        self.bn31 = nn.BatchNorm2d(100)
        self.pool31 = nn.MaxPool2d((1,3),stride = (1,3))
        #block4
        self.conv41 = nn.Conv2d(1,200,(100,10)) # after permute 1, 200*40
        self.bn41 = nn.BatchNorm2d(200)
        self.pool41= nn.MaxPool2d((1,3),stride = (1,3))
        #linear 
        self.fc1 = nn.Linear(200*7,600)
        self.fc2 = nn.Linear(600,60)
        self.fc3 = nn.Linear(60,4)
        self.drop = nn.Dropout(0.8)
 
    def forward(self,x):
        x = F.relu(self.bn11(self.conv11(x)))
        x = F.relu(self.bn12(self.conv12(x)))
        x = F.relu(self.bn13(self.conv13(x)))
        x = F.relu(self.bn14(self.conv14(x)))
        #x = F.relu(self.conv11(x))
        #x = F.relu(self.conv12(x))
        #x = F.relu(self.conv13(x))
        #x = F.relu(self.bn14(self.conv14(x)))
        x = self.pool1(x)
        #print(x.size())
        x = x.permute(0,2,1,3)
        print(list(x.size())[2])
        print(x.shape)
        x = x.view(list(x.size())[0],list(x.size())[2],list(x.size())[3])
        print(x.shape)
        #x = x.reshape()
        #print(x.size())
        
        
        # now do normal 2d [conv - max] * 3
        x = F.relu(self.bn21(self.conv21(x)))
        #x = F.relu(self.conv21(x))
        x = x.permute(0,2,1,3)
        x = self.pool21(x)
        #print('after block 2',x.size())
        
        x = F.relu(self.bn31(self.conv31(x)))
        #x = F.relu(self.conv31(x))
        x = x.permute(0,2,1,3)
        x = self.pool31(x)
        #print(x.size())
        
        x = F.relu(self.bn41(self.conv41(x)))
        #x = F.relu(self.conv41(x))
        x = x.permute(0,2,1,3)
        x = self.pool41(x)
       # print(x.size())
        
        x = x.view(-1,200*7)
        x = F.elu(self.fc1(x))
        x = self.drop(F.elu(self.fc2(x)))
        x = self.fc3(x)
        
        return x
net = Net()
print(net)
'''

"\nclass Net(nn.Module):\n    def __init__(self):\n        super(Net,self).__init__()  # initial the model\n        self.conv11 = nn.Conv2d(1,25,(1,10)) # depth 25, 22*990\n        self.bn11 = nn.BatchNorm2d(25)\n        # permute 990,  22 x 12\n        self.conv12 = nn.Conv2d(25,25,kernel_size = (2,5),stride = (1,1)) # 25, 21 x 986\n        self.bn12 = nn.BatchNorm2d(25) \n        \n        self.conv13 = nn.Conv2d(25,25,kernel_size = (4,3),stride = (1,1)) # 25, 18 x 984\n        self.bn13 = nn.BatchNorm2d(25) \n        \n        self.conv14 = nn.Conv2d(25,25,kernel_size = (18,2),stride = (1,1)) # 25, 1 x 983\n        self.bn14 = nn.BatchNorm2d(25) \n        # permute 0, 2,1,3 ----> N, 1,25,983\n        self.pool1 = nn.MaxPool2d((1,3),stride = (1,3)) # 1, 25*328\n        # now do normal conv - maxpool\n        # block2\n        # after permute, starts at 1,25,328\n        self.conv21 = nn.Conv2d(1,50,kernel_size = (25,10),stride = (1,1)) # 50,1,319\n        self.bn21 = nn.BatchNorm2d(5

In [140]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet,self).__init__()  # initial the model
        self.conv11 = nn.Conv2d(1,30,(1,10)) # depth 30, 22*991
        self.bn11 = nn.BatchNorm2d(30)
        # permute 990,  22 x 12
        self.conv12 = nn.Conv2d(30,40,kernel_size = (2,5),stride = (1,1)) # 25, 21 x 987
        self.bn12 = nn.BatchNorm2d(40) 
        
        self.conv13 = nn.Conv2d(40,40,kernel_size = (5,3),stride = (1,1)) # 25, 17 x 985
        self.bn13 = nn.BatchNorm2d(40) 
        
        self.conv14 = nn.Conv2d(40,40,kernel_size = (17,2),stride = (1,1)) # 25, 1 x 984
        self.bn14 = nn.BatchNorm2d(40) 
        # permute 0, 2,1,3 ----> N, 1,25,984
        self.pool1 = nn.MaxPool2d((1,4),stride = (1,2)) # 1, 40*491
 
    def forward(self,x):
        #x = torch.Tensor(x.numpy().transpose(0,2,1))
        x = F.relu(self.bn11(self.conv11(x)))
        x = F.relu(self.bn12(self.conv12(x)))
        x = F.relu(self.bn13(self.conv13(x)))
        x = F.relu(self.bn14(self.conv14(x)))

        x = self.pool1(x)

        x = x.permute(0,2,1,3)

        x = x.view(list(x.size())[0],list(x.size())[2],list(x.size())[3])
        # N, 40,492
        x = torch.Tensor(x.detach().numpy().transpose(0,2,1))
        return x

In [141]:
# 两层CNN + 三层LSTM
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim):
        super(LSTM, self).__init__()
        #self.conv = convnet  #这个是用已经训练过得到的CNN
        self.conv = ConvNet()  #这个是用没有训练过的CNN
        self.LSTM1 = nn.LSTM(input_dim, hidden_dim1, bidirectional = True, num_layers=1, batch_first = True, dropout = 0)
        self.LSTM2 = nn.LSTM(hidden_dim1*2, hidden_dim2, bidirectional = True, batch_first = True)
        self.LSTM3 = nn.LSTM(hidden_dim2*2, hidden_dim3, bidirectional = True, batch_first = True)
        self.fc1 = nn.Linear(hidden_dim3*2*491, 1000) #要取决于用那种数据，249对应未处理的数据，124对应嵘哥的downsample数据（嵘哥ds！）
        self.bn1 = nn.BatchNorm1d(1000)
        self.drop1 = nn.Dropout(0.9)
        self.fc2 = nn.Linear(1000,100)
        self.bn2 = nn.BatchNorm1d(100)
        self.fc3 = nn.Linear(100,output_dim)
        
    
    def forward(self, x, h=None, c=None):
        x = self.conv(x)
        if type(h) == type(None) and type(c) == type(None):
            out, (hn, cn) = self.LSTM1(x)
            out, (hn, cn) = self.LSTM2(out)
            out, (hn, cn) = self.LSTM3(out)
        else:
            out, (hn, cn) = self.LSTM1(x, h.detach(), c.detach())
            out, (hn, cn) = self.LSTM2(out, h.detach(), c.detach())
            out, (hn, cn) = self.LSTM3(out, h.detach(), c.detach())
        #out = self.drop1(F.relu(self.bn1(self.fc1(out[:, -1, :]))))
        out = self.drop1(F.relu(self.bn1(self.fc1(out.reshape(out.shape[0],-1)))))
        out = self.drop1(F.relu(self.bn2(self.fc2(out))))
        out = self.fc3(out)
        
        return out

In [142]:
input_dim = 40
hidden_dim1 = 40
hidden_dim2 = 40
hidden_dim3 = 40
output_dim = 4
model = LSTM(input_dim, hidden_dim1, hidden_dim2, hidden_dim3, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(),alpha = 0.99, lr=0.001, weight_decay = 0.01)

In [143]:
val_acc_history = []
train_acc_history = []
train_loss_history = []
val_loss_history = []
t0 = time.time()
num_epochs = 100
for epoch in range(num_epochs):
    tstart = time.time()
    for i, data in enumerate(train_loader_1):
        inputs, labels = data
        #inputs = convnet(inputs)
        #inputs = torch.Tensor(inputs.detach().numpy().transpose(0,2,1)) #249*60
        model.train()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _,predicted = torch.max(outputs.data, 1)
        train_acc = (predicted == labels).sum().item() / len(labels)
    model.eval()
    train_correct, train_total = 0, 0
    train_loss = 0
    for train_data in train_loader_1:
        train_inputs, train_labels = train_data
        #train_inputs = convnet(train_inputs)
        #train_inputs = torch.Tensor(train_inputs.detach().numpy().transpose(0,2,1)) #249*60
        train_outputs = model(train_inputs)
        _, train_predicted = torch.max(train_outputs.data, 1)
        train_total += train_labels.size(0)
        train_correct += (train_predicted == train_labels).sum().item()
        train_loss += criterion(train_outputs, train_labels).item()
    train_acc = train_correct / train_total
    train_acc_history.append(train_acc)
    train_loss_history.append(train_loss)
        
    #pXtrain = model(Xtrain)
    #ptrain = torch.argmax(pXtrain, axis = 1)
    #train_acc = np.mean(ptrain.numpy() == ytrain.numpy())
    #train_accs.append(train_acc)
    #tloss = criterion(pXtrain, ytrain)
    #train_losses.append(tloss.item())
    
    val_correct, val_total = 0, 0
    val_loss = 0
    for val_data in val_loader_1:
        val_inputs, val_labels = val_data
        #val_inputs = convnet(val_inputs)
        #val_inputs = torch.Tensor(val_inputs.detach().numpy().transpose(0,2,1)) #249*60
        val_outputs = model(val_inputs)
        _, val_predicted = torch.max(val_outputs.data, 1)
        val_total += val_labels.size(0)
        val_correct += (val_predicted == val_labels).sum().item()
        val_loss += criterion(val_outputs, val_labels).item()
    val_acc = val_correct / val_total
    val_acc_history.append(val_acc)
    val_loss_history.append(val_loss)
    
    #pXval = model(Xval)
    #pval = torch.argmax(pXval, axis = 1)
    #val_acc = np.mean(pval.numpy() == yval.numpy())
    #val_accs.append(val_acc)
    #vloss = criterion(pXval, yval)
    #val_losses.append(vloss.item())
    tend = time.time()
    print('epoch: {:<3d}    time: {:<3.2f}    train_loss: {:<3.3f}    train acc: {:<1.3f}    val_loss: {:<3.3f}    val acc: {:<1.3f}'.format(epoch+1, 
            tend - tstart, train_loss, train_acc, val_loss, val_acc))
time_total = time.time() - t0
print('Total time: {:4.3f} seconds, average time per epoch: {:4.3f}'.format(time_total, time_total / num_epochs))

test_correct, test_total = 0, 0
for test_data in test_loader_1:
    test_inputs, test_labels = test_data
    #test_inputs = convnet(test_inputs)
    #test_inputs = torch.Tensor(test_inputs.detach().numpy().transpose(0,2,1)) #249*60
    test_outputs = model(test_inputs)
    _, test_predicted = torch.max(test_outputs.data, 1)
    test_total += test_labels.size(0)
    test_correct += (test_predicted == test_labels).sum().item()
test_acc = test_correct / test_total
print('Test accuracy is: ',test_acc)

epoch: 1      time: 26.32    train_loss: 8.226    train acc: 0.344    val_loss: 5.619    val acc: 0.229
epoch: 2      time: 25.90    train_loss: 8.033    train acc: 0.328    val_loss: 5.714    val acc: 0.229
epoch: 3      time: 25.51    train_loss: 7.747    train acc: 0.413    val_loss: 5.577    val acc: 0.292
epoch: 4      time: 25.44    train_loss: 7.566    train acc: 0.450    val_loss: 5.682    val acc: 0.375
epoch: 5      time: 25.36    train_loss: 7.440    train acc: 0.487    val_loss: 5.436    val acc: 0.354
epoch: 6      time: 25.30    train_loss: 7.385    train acc: 0.508    val_loss: 5.475    val acc: 0.396
epoch: 7      time: 25.31    train_loss: 7.158    train acc: 0.587    val_loss: 5.670    val acc: 0.271
epoch: 8      time: 25.33    train_loss: 7.155    train acc: 0.614    val_loss: 5.670    val acc: 0.292
epoch: 9      time: 25.41    train_loss: 6.955    train acc: 0.688    val_loss: 5.635    val acc: 0.271
epoch: 10     time: 25.27    train_loss: 6.972    train acc: 0.6

KeyboardInterrupt: 